In [10]:
# Import necessary libraries
import nest_asyncio
import asyncio
import aiohttp
from unittest.mock import patch, AsyncMock
import importlib.util

# Allow nested event loops for Jupyter
nest_asyncio.apply()

# Dynamically import the serverless_utils module
spec = importlib.util.spec_from_file_location(
    "serverless_utils", "D:/TradingRobotPlug2/Scripts/Utilities/serverless_utils.py"
)
serverless_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(serverless_utils)

# Access the ServerlessFetcher class
ServerlessFetcher = serverless_utils.ServerlessFetcher

# Set up test data
mock_response_data = {"status": "success", "data": {"symbol": "AAPL", "price": 150.0}}

# Initialize the ServerlessFetcher instance
fetcher = ServerlessFetcher()

# Define the test coroutine
async def test_fetch_interactively():
    """
    Test the ServerlessFetcher.fetch() method interactively with mocks.
    """
    # Mock the environment variable
    with patch("os.getenv", return_value="https://example-serverless-api.com/fetch"):
        # Mock aiohttp.ClientSession
        with patch("aiohttp.ClientSession") as MockClientSession:
            # Mock response object
            mock_response = AsyncMock()
            mock_response.status = 200
            mock_response.json = AsyncMock(return_value=mock_response_data)

            # Mock session and its post method
            mock_session = AsyncMock()
            mock_session.post.return_value.__aenter__.return_value = mock_response
            MockClientSession.return_value.__aenter__.return_value = mock_session

            # Call the fetch method
            print("Calling fetch()...")
            result = await fetcher.fetch(symbol="AAPL", data_sources=["Alpha Vantage"])
            print("Result:", result)

            # Debug: Check if mock_session.post was called
            if not mock_session.post.called:
                print("Debug: mock_session.post was never called!")
            else:
                print("Debug: mock_session.post call args:", mock_session.post.call_args)

            # Confirm mocked behavior
            mock_session.post.assert_called_once_with(
                "https://example-serverless-api.com/fetch",
                json={"symbol": "AAPL", "data_sources": ["Alpha Vantage"], "kwargs": {}}
            )


# Run the test coroutine (Jupyter/Interactive-safe way)
await test_fetch_interactively()


Calling fetch()...
Result: {}
Debug: mock_session.post was never called!


AssertionError: Expected 'post' to be called once. Called 0 times.